# Time Domain Sphere Representation:
### this notebook takes the IMU samples from: https://fpv.ifi.uzh.ch/datasets/, and shows the represntation  of it on a sphere with a radius of 1.

#### Download the zip file from the site, and extract the needed data.

In [1]:
import requests
import zipfile
import io
import numpy as np

# Function to download the zip file
def download_zip(url, save_path):
    response = requests.get(url)
    with open(save_path, 'wb') as f:
        f.write(response.content)

# Function to read data from 'imu.txt' into arrays for each column
def read_imu_data(zip_path):
    serial_num = []
    timestamps = []
    wx_values = []
    wy_values = []
    wz_values = []
    ax_values = []
    ay_values = []
    az_values = []
    
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        with zip_ref.open('imu.txt') as imu_file:
            next(imu_file)  # Skip the first line
            for line in imu_file:
                line = line.decode('utf-8').strip().split()
                sn, ts, wx, wy, wz, ax, ay, az = map(float, line)
                serial_num.append(sn)
                timestamps.append(ts)
                wx_values.append(wx)
                wy_values.append(wy)
                wz_values.append(wz)
                ax_values.append(ax)
                ay_values.append(ay)
                az_values.append(az)
    
    # Convert lists to numpy arrays
    serial_num = np.array(serial_num)
    timestamps = np.array(timestamps)
    wx_values = np.array(wx_values)
    wy_values = np.array(wy_values)
    wz_values = np.array(wz_values)
    ax_values = np.array(ax_values)
    ay_values = np.array(ay_values)
    az_values = np.array(az_values)
    
    return serial_num, timestamps, wx_values, wy_values, wz_values, ax_values, ay_values, az_values

# URL of the zip file
zip_url = 'http://rpg.ifi.uzh.ch/datasets/uzh-fpv-newer-versions/v3/indoor_forward_9_davis_with_gt.zip'
#'http://rpg.ifi.uzh.ch/datasets/uzh-fpv-newer-versions/v3/indoor_forward_3_davis_with_gt.zip'

# Path to save the downloaded zip file
zip_save_path = 'indoor_forward_3_davis_with_gt.zip'

# Download the zip file
download_zip(zip_url, zip_save_path)

# Read data from the 'imu.txt' file into arrays for each column
samp_num, timestamps, wx_values, wy_values, wz_values, ax_values, ay_values, az_values = read_imu_data(zip_save_path)


#### Show the representation of the sample on the sphere: 
##### Linear Acceleration:

In [2]:
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

def plot_sphere(ax, ay, az):
    # Normalize the acceleration data
    norm = np.sqrt(ax**2 + ay**2 + az**2)
    ax_norm = ax / norm
    ay_norm = ay / norm
    az_norm = az / norm

    # Create a figure and 3D axis
    fig = plt.figure()
    ax_plot = fig.add_subplot(111, projection='3d')

    # Plot the sphere
    u = np.linspace(0, 2 * np.pi, 100)
    v = np.linspace(0, np.pi, 100)
    x = np.outer(np.cos(u), np.sin(v))
    y = np.outer(np.sin(u), np.sin(v))
    z = np.outer(np.ones(np.size(u)), np.cos(v))

    # Plot the normalized acceleration samples on the sphere
    ax_plot.scatter(ax_norm, ay_norm, az_norm, c='r', marker='o')

    # Connect the acceleration samples with lines
    for i in range(len(ax_norm) - 1):
        ax_plot.plot([ax_norm[i], ax_norm[i + 1]], [ay_norm[i], ay_norm[i + 1]], [az_norm[i], az_norm[i + 1]], color='r', linestyle='-', linewidth=2)

    # Plot the sphere wireframe
    ax_plot.plot_wireframe(x, y, z, color='k', alpha=0.1)

    # Set axis labels and title
    ax_plot.set_xlabel('X')
    ax_plot.set_ylabel('Y')
    ax_plot.set_zlabel('Z')
    ax_plot.set_title('Acceleration Data on a Unit Sphere')

    # Adjust the aspect ratio and view angle
    max_radius = 1.2
    ax_plot.auto_scale_xyz([-max_radius, max_radius], [-max_radius, max_radius], [-max_radius, max_radius])
    ax_plot.view_init(elev=30, azim=-45)

    plt.show()

# Example usage
ax = ax_values[::100]
ay = ay_values[::100]
az = az_values[::100]

plot_sphere(ax, ay, az)

##### Angular Acceleration

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

def plot_sphere(wx, wy, wz, radius=1):
    # Normalize the angular acceleration data
    norm = np.sqrt(wx**2 + wy**2 + wz**2)
    wx_norm = wx / norm
    wy_norm = wy / norm
    wz_norm = wz / norm

    # Create a new figure and 3D axis
    fig = plt.figure(figsize=(10, 8))
    ax_plot = fig.add_subplot(111, projection='3d')

    # Plot the sphere
    u = np.linspace(0, 2 * np.pi, 100)
    v = np.linspace(0, np.pi, 100)
    x = radius * np.outer(np.cos(u), np.sin(v))
    y = radius * np.outer(np.sin(u), np.sin(v))
    z = radius * np.outer(np.ones(np.size(u)), np.cos(v))

    # Plot the normalized angular acceleration samples on the sphere
    ax_plot.scatter(wx_norm * radius, wy_norm * radius, wz_norm * radius, c='r', marker='o')

    # Connect the angular acceleration samples with lines
    for i in range(len(wx_norm) - 1):
        ax_plot.plot([wx_norm[i] * radius, wx_norm[i + 1] * radius], [wy_norm[i] * radius, wy_norm[i + 1] * radius], [wz_norm[i] * radius, wz_norm[i + 1] * radius], color='b', linestyle='-', linewidth=1)

    # Plot the sphere wireframe
    ax_plot.plot_wireframe(x, y, z, color='k', alpha=0.2)

    # Set axis labels and title
    ax_plot.set_xlabel('X')
    ax_plot.set_ylabel('Y')
    ax_plot.set_zlabel('Z')
    ax_plot.set_title(f'Angular Acceleration Data on a Sphere (Radius = {radius})')

    # Adjust the aspect ratio and view angle
    max_radius = radius * 1.2
    ax_plot.auto_scale_xyz([-max_radius, max_radius], [-max_radius, max_radius], [-max_radius, max_radius])
    ax_plot.view_init(elev=30, azim=-45)

    # Display the plot in a separate window
    plt.show()

# Example usage
wx = wx_values[::100]
wy = wx_values[::100]
wz = wz_values[::100]
radius = 1  # Set the desired radius value

plot_sphere(wx, wy, wz, radius)